<a href="https://colab.research.google.com/github/Skriller18/JungliFungli/blob/main/ASL_Hackathon(8th_Mile).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#Splitting into test and train dataset

'''import os
import random
import shutil
import string

def split_dataset(input_dir, split_ratio):
    os.makedirs(train_dir, exist_ok=True)
    os.makedirs(test_dir, exist_ok=True)

    for filename in os.listdir(input_dir):
        filepath = os.path.join(input_dir, filename)
        if random.random() < split_ratio:
            shutil.copy(filepath, os.path.join(train_dir, filename))
        else:
            shutil.copy(filepath, os.path.join(test_dir, filename))'''

'import os\nimport random\nimport shutil\nimport string\n\ndef split_dataset(input_dir, split_ratio):\n    os.makedirs(train_dir, exist_ok=True)\n    os.makedirs(test_dir, exist_ok=True)\n\n    for filename in os.listdir(input_dir):\n        filepath = os.path.join(input_dir, filename)\n        if random.random() < split_ratio:\n            shutil.copy(filepath, os.path.join(train_dir, filename))\n        else:\n            shutil.copy(filepath, os.path.join(test_dir, filename))'

In [ ]:
'''input = "/content/drive/MyDrive/asl_dataset/"
output = "/content/drive/MyDrive/Sign_Dataset"
for char in string.ascii_lowercase:
  input1 = os.path.join(input,char)
  train = "/Train/"+char
  test = "/Test/"+char
  train_dir = os.path.join(output,train)
  test_dir = os.path.join(output,test)
  split_dataset(input1,0.8)'''

'input = "/content/drive/MyDrive/asl_dataset/"\noutput = "/content/drive/MyDrive/Sign_Dataset"\nfor char in string.ascii_lowercase:\n  input1 = os.path.join(input,char)\n  train = "/Train/"+char\n  test = "/Test/"+char\n  train_dir = os.path.join(output,train)\n  test_dir = os.path.join(output,test)\n  split_dataset(input1,0.8)'

In [ ]:
import tensorflow as tf
import matplotlib.pyplot as plt

In [ ]:
!wget https://raw.githubusercontent.com/mrdbourke/tensorflow-deep-learning/main/extras/helper_functions.py

--2023-06-15 18:04:13--  https://raw.githubusercontent.com/mrdbourke/tensorflow-deep-learning/main/extras/helper_functions.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 10246 (10K) [text/plain]
Saving to: ‘helper_functions.py’

helper_functions.py 100%[===================>]  10.01K  --.-KB/s    in 0s      

2023-06-15 18:04:13 (75.4 MB/s) - ‘helper_functions.py’ saved [10246/10246]



In [ ]:
from helper_functions import create_tensorboard_callback, plot_loss_curves, unzip_data, walk_through_dir

In [ ]:
walk_through_dir("/content/drive/MyDrive/Sign_Dataset")

There are 2 directories and 0 images in '/content/drive/MyDrive/Sign_Dataset'.
There are 26 directories and 0 images in '/content/drive/MyDrive/Sign_Dataset/Test'.
There are 0 directories and 46 images in '/content/drive/MyDrive/Sign_Dataset/Test/b'.
There are 0 directories and 50 images in '/content/drive/MyDrive/Sign_Dataset/Test/a'.
There are 0 directories and 39 images in '/content/drive/MyDrive/Sign_Dataset/Test/c'.
There are 0 directories and 53 images in '/content/drive/MyDrive/Sign_Dataset/Test/e'.
There are 0 directories and 52 images in '/content/drive/MyDrive/Sign_Dataset/Test/f'.
There are 0 directories and 44 images in '/content/drive/MyDrive/Sign_Dataset/Test/j'.
There are 0 directories and 46 images in '/content/drive/MyDrive/Sign_Dataset/Test/d'.
There are 0 directories and 44 images in '/content/drive/MyDrive/Sign_Dataset/Test/i'.
There are 0 directories and 42 images in '/content/drive/MyDrive/Sign_Dataset/Test/h'.
There are 0 directories and 45 images in '/content/dr

In [ ]:
test_dir = "/content/drive/MyDrive/Sign_Dataset/Test"
train_dir = "/content/drive/MyDrive/Sign_Dataset/Train"

In [ ]:
IMG_SIZE = (224, 224)
BATCH_SIZE = 32

train_data = tf.keras.preprocessing.image_dataset_from_directory(directory = train_dir,
                                                                  image_size = IMG_SIZE,
                                                                  label_mode = "categorical",
                                                                  batch_size = BATCH_SIZE)

test_data = tf.keras.preprocessing.image_dataset_from_directory(directory = test_dir,
                                                                image_size = IMG_SIZE,
                                                                label_mode = "categorical",
                                                                batch_size = BATCH_SIZE)

Found 1815 files belonging to 26 classes.
Found 1198 files belonging to 26 classes.


In [ ]:
#Creating the base model with tf.keras.functional API
base_model = tf.keras.applications.EfficientNetB0(include_top=False)

#Freeze the base model
base_model.trainable=False

#Create inputs into the model
inputs = tf.keras.layers.Input(shape = (224,224,3), name = "Input_Layer")

#Normalising(incase for Resnet)
#normal = tf.keras.layers.experimental.preprocessing.Rescaling(1./255)

#Pass the inputs to the base model
x = base_model(inputs)

#Average pool the outputs
x = tf.keras.layers.GlobalAveragePooling2D(name="golbal_pooling_layer")(x)
print(f"Shape after GLOBALAVERAGEPOOLING: {x.shape}")

#Create the output activation layer
outputs = tf.keras.layers.Dense(26,activation="softmax",name="output_layer")(x)

#Combine inputs and outputs
model_0 = tf.keras.Model(inputs,outputs)

#Compile the model
model_0.compile(loss="categorical_crossentropy",
                optimizer=tf.keras.optimizers.Adam(),
                metrics=["accuracy"])

#Fit the model
history_0 = model_0.fit(train_data,
                        epochs = 5,
                        steps_per_epoch = len(train_data),
                        validation_data = test_data,
                        validation_steps = int(0.25*len(test_data)),
                        callbacks = [create_tensorboard_callback(dir_name = "Sign_dataset",
                                                                 experiment_name="ASL-Dataset")])

16705208/16705208 [==============================] - 0s 0us/step


Shape after GLOBALAVERAGEPOOLING: (None, 1280)
Saving TensorBoard log files to: Sign_dataset/ASL-Dataset/20230615-180431
Epoch 1/5
57/57 [==============================] - 241s 4s/step - loss: 2.6611 - accuracy: 0.3620 - val_loss: 1.9377 - val_accuracy: 0.7847
Epoch 2/5


In [ ]:
model_0.evaluate(test_data)

In [ ]:
plot_loss_curves(history_0)

In [ ]:
model_0.summary()

In [ ]:
base_model.summary()

In [ ]:
'''Y_probs = multimod.predict(test_data)
print(Y_probs)

Y_preds = Y_probs.argmax(axis=1)
print(Y_preds)'''

In [ ]:
'''#Plotting our predictions in a confusion matrix
import itertools
from sklearn.metrics import confusion_matrix
figsize = (10, 10)

def make_confusion_matrix(y_true, y_pred, classes=None, figsize=(10, 10), text_size=15):
#Create the confusion matrix
  cm = confusion_matrix(y_true, tf.round(y_pred))
  cm_norm = cm.astype("float")/ cm.sum(axis=1)[:,np.newaxis]
  n_classes = cm.shape[0]

  #Subplotting each class and associating it with each other
  fig, ax = plt.subplots(figsize=figsize)

  #Create a matrix plot
  cax = ax.matshow(cm, cmap=plt.cm.Blues)
  fig.colorbar(cax)

  #Set class labels
  if classes:
    labels=classes
  else:
    labels=np.arange(cm.shape[0])

  #Label the axes
  ax.set(title="Confusion Matrix",
        xlabel="Predicted Label",
        ylabel="True Label",
        xticks=np.arange(n_classes),
        yticks=np.arange(n_classes),
        xticklabels=labels,
        yticklabels=labels)

  #Set threshold for different colors
  threshold = (cm.max()+ cm.min())/2

  #Plot text on each cell
  for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[0])):
    plt.text(j, i, f"{cm[i,j]} ({cm_norm[i, j]*100:.1f}%)",
            horizontalalignment="center",
            color="white" if cm[i, j] > threshold else "black",
            size=15)'''

In [ ]:
#Create a function to import an image and resize it to able to use it for the model and predict results

def load_and_prep_img(filename, img_shape=24):
  #Read the image
  img = tf.io.read_file(filename)
  #Decode the read file into a tensor
  img = tf.image.decode_image(img)
  #Resize our image
  img = tf.image.resize(img, size={img_shape, img_shape})
  #Normalising the image
  img = img/255
  return

In [ ]:
from IPython.display import display, Javascript
from google.colab.output import eval_js
from base64 import b64decode

def take_photo(filename='photo.jpg', quality=0.8):
  js = Javascript('''
    async function takePhoto(quality) {
      const div = document.createElement('div');
      const capture = document.createElement('button');
      capture.textContent = 'Capture';
      div.appendChild(capture);

      const video = document.createElement('video');
      video.style.display = 'block';
      const stream = await navigator.mediaDevices.getUserMedia({video: true});

      document.body.appendChild(div);
      div.appendChild(video);
      video.srcObject = stream;
      await video.play();

      // Resize the output to fit the video element.
      google.colab.output.setIframeHeight(document.documentElement.scrollHeight, true);

      // Wait for Capture to be clicked.
      await new Promise((resolve) => capture.onclick = resolve);

      const canvas = document.createElement('canvas');
      canvas.width = video.videoWidth;
      canvas.height = video.videoHeight;
      canvas.getContext('2d').drawImage(video, 0, 0);
      stream.getVideoTracks()[0].stop();
      div.remove();
      return canvas.toDataURL('image/jpeg', quality);
    }
    ''')
  display(js)
  data = eval_js('takePhoto({})'.format(quality))
  binary = b64decode(data.split(',')[1])
  with open(filename, 'wb') as f:
    f.write(binary)
  return filename

In [ ]:
from IPython.display import Image
try:
  filename = take_photo()
  print('Saved to {}'.format(filename))

  # Show the image which was just taken.
  display(Image(filename))
except Exception as err:
  # Errors will be thrown if the user does not have a webcam or if they do not
  # grant the page permission to access it.
  print(str(err))

In [ ]:
def load_and_prep_image(filename, img_shape=224):
  # Read in target file (an image)
  img = tf.io.read_file(filename)

  # Decode the read file into a tensor & ensure 3 colour channels
  # (our model is trained on images with 3 colour channels and sometimes images have 4 colour channels)
  img = tf.image.decode_image(img, channels=3)

  # Resize the image (to the same size our model was trained on)
  img = tf.image.resize(img, size = [img_shape, img_shape])

  # Rescale the image (get all values between 0 and 1)
  img = img/255
  return img

In [ ]:
import numpy as np
def pred_and_plot(model, filename, class_names):
  # Import the target image and preprocess it
  img = load_and_prep_image(filename)

  # Make a prediction
  pred = model.predict(tf.expand_dims(img, axis=0))

  # Get the predicted class
  pred_array = np.array(pred)
  pred_class = class_names[np.argmax(pred_array)]

  # Plot the image and predicted class
  plt.imshow(img)
  plt.title(f"Prediction: {pred_class}")
  plt.axis(False);

In [ ]:
class_names = ['a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z']

In [ ]:
pred_and_plot(model_0,"/content/drive/MyDrive/p.jpg", class_names)